## Non-Linear Model Comparison

This notebook evaluates whether non-linear models can outperform the frozen class-weighted Logistic Regression baseline for flight delay prediction. All comparisons use the same feature set, data split, and evaluation metrics to ensure a fair assessment.


In [2]:
import numpy as np
import pandas as pd

In [3]:
X_train_scaled = pd.read_csv("../data/processed/X_train_scaled.csv")
X_test_scaled = pd.read_csv("../data/processed/X_test_scaled.csv")

y_train = pd.read_csv("../data/processed/y_train.csv").squeeze()
y_test = pd.read_csv("../data/processed/y_test.csv").squeeze()

feature_names = X_train_scaled.columns


## WE use Random Forest for Now

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    roc_auc_score
)


In [5]:
rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=12,
    min_samples_leaf=50,
    n_jobs=-1,
    random_state=42
)


In [6]:
rf.fit(X_train_scaled, y_train)


,n_estimators,200
,criterion,'gini'
,max_depth,12
,min_samples_split,2
,min_samples_leaf,50
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [7]:
y_pred_rf = rf.predict(X_test_scaled)
y_proba_rf = rf.predict_proba(X_test_scaled)[:, 1]


In [8]:
print(confusion_matrix(y_test, y_pred_rf))

[[856143  12908]
 [ 94137  62021]]


In [9]:
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

         0.0       0.90      0.99      0.94    869051
         1.0       0.83      0.40      0.54    156158

    accuracy                           0.90   1025209
   macro avg       0.86      0.69      0.74   1025209
weighted avg       0.89      0.90      0.88   1025209



In [11]:
roc_auc_rf = roc_auc_score(y_test, y_proba_rf)
roc_auc_rf

0.8201784705469534

### Random Forest Baseline Evaluation

A baseline Random Forest model was trained using conservative hyperparameters to capture potential non-linear interactions in the data. The model achieved a ROC–AUC of approximately **0.82**, indicating stronger class separation compared to the Logistic Regression baseline.

At the default decision threshold of 0.5, the Random Forest exhibited **high precision** for delayed flights, meaning that when a delay was predicted, it was usually correct. However, recall for delayed flights remained relatively low, indicating that a significant number of delays were still missed. This conservative behavior is typical of Random Forest models when used with default thresholds, as they tend to produce sharper probability estimates.


In [12]:
import joblib

joblib.dump(
    rf,
    "../models/random_forest_baseline.joblib"
)


['../models/random_forest_baseline.joblib']

### Comparison with Logistic Regression Baseline

Compared to the frozen class-weighted Logistic Regression model, the Random Forest demonstrates superior ranking performance, as reflected by its higher ROC–AUC score. This suggests that non-linear interactions between features contribute additional predictive signal.

However, in terms of operational behavior, the two models differ substantially. The Logistic Regression model prioritizes recall and identifies a larger proportion of delayed flights, while the Random Forest model favors precision and issues fewer false delay warnings. Neither model dominates across all metrics, highlighting a trade-off between capturing more delays and avoiding false alarms.


### Key Insight

The comparison reveals that while non-linear models improve overall class separation, decision-making performance remains strongly dependent on the chosen probability threshold. As with Logistic Regression, threshold tuning is required to align Random Forest predictions with operational priorities. This reinforces the importance of separating model learning from decision policy when evaluating predictive systems.


### Threshold evaluation

In [14]:
from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate_threshold(y_true, y_proba, threshold):
    y_pred = (y_proba >= threshold).astype(int)
    return {
        "threshold": threshold,
        "precision": precision_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred)
    }

#For Random Forest, we typically test lower thresholds:    
thresholds = [0.50, 0.45, 0.40, 0.35, 0.30]

rf_threshold_results = [
    evaluate_threshold(y_test, y_proba_rf, t)
    for t in thresholds
]

import pandas as pd
pd.DataFrame(rf_threshold_results)


,threshold,precision,recall,f1
0,0.50,0.827730,0.397168,0.536776
1,0.45,0.800973,0.421573,0.552402
2,0.40,0.752657,0.460719,0.571568
3,0.35,0.697400,0.499302,0.581955
4,0.30,0.646290,0.532621,0.583975


### Random Forest Threshold Tuning

Threshold tuning was applied to the Random Forest model to adjust its conservative default behavior. As the decision threshold was lowered, recall for delayed flights increased steadily while precision decreased gradually. The F1-score improved consistently across the evaluated thresholds, indicating that the model’s probability ranking was strong and benefited from a more permissive decision boundary.

A threshold of **0.30** was selected as the optimal operating point, achieving higher recall and precision than the Logistic Regression baseline while also yielding the highest F1-score among the tested thresholds.


### Final Model Comparison

After threshold tuning, the Random Forest model outperformed the frozen class-weighted Logistic Regression baseline across all key metrics. The Random Forest achieved higher recall and precision for delayed flights, along with a substantially higher ROC–AUC score, indicating superior class separation. This suggests that non-linear interactions between operational, temporal, and weather-related features provide meaningful additional predictive signal beyond what a linear model can capture.


In [15]:
import joblib

joblib.dump(
    rf,
    "../models/random_forest_final.joblib"
)


['../models/random_forest_final.joblib']

In [16]:
final_rf_threshold = 0.30

joblib.dump(
    final_rf_threshold,
    "../models/random_forest_final_threshold.joblib"
)


['../models/random_forest_final_threshold.joblib']

In [17]:
joblib.dump(
    X_train_scaled.columns.tolist(),
    "../models/random_forest_feature_list.joblib"
)


['../models/random_forest_feature_list.joblib']

## Final Model Selection

After evaluating Logistic Regression and Random Forest models under consistent preprocessing, data splits, and evaluation metrics, the Random Forest model was selected as the final model for flight delay prediction.

Threshold tuning revealed that a decision threshold of **0.30** provides the optimal balance between recall and precision for delayed flights. At this operating point, the Random Forest model achieved superior recall, precision, F1-score, and ROC–AUC compared to the class-weighted Logistic Regression baseline.

This result indicates that non-linear interactions among operational, temporal, and weather-related features contribute meaningful predictive signal beyond what a linear model can capture. The Random Forest model with a threshold of 0.30 is therefore frozen as the final model for this project.
